In [ ]:
#10 M events were generated at PX/NX/PY/NY/PZ/NZ positon including the isotopes 'Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208'
#skipEThreshold: 0.1
#Each isotopes 40 seeds;each simulation: 250K events
# 38 failed for Th228 and 27 failed for Po216
#Total root files in g4 215 for PX , 215 for NX
#Total root files in recon 211
#comparing with the result in the 5 th slide
#https://docs.google.com/viewer?url=https%3A%2F%2Fnexowiki.llnl.gov%2Fimages%2Fa%2Fa1%2FScott_Schwartz_Th228_Location_Reconstruction_Study.pdf

In [ ]:
#libraries
import uproot
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
import scienceplots
plt.rcParams['figure.figsize'] = [30, 20]
sns.set(rc={'figure.figsize':(30,20)})
#sns.set(font_scale=1.0)
sns.set_theme(style="darkgrid")

# DATA FILES

In [ ]:
file_px='/home/thakur/slac_data/10Mth228/10Mth228_Th228_pos648.5_0_-1022.6.root' #PX location
file_nx='/home/thakur/slac_data/s1/s1_Th228_pos-648.5_0_-1022.6.root'            #NX location
skipEThreshold=0.1

# FILTERS

In [ ]:
#cuts
apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'


dec102020_filter='''
standoff>100 &\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216)) &\
~NESTBugFound &\
~NearAnodeBugFound &\
m_DNNvalue>0.85)
'''
ja_filter='(standoff>100) & m_DNNvalue>0.85 & passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < (1.077 * m_nQ + 313)) & (m_nOPCal > (0.597 * m_nQ - 216))& (abs(energy-2614)<10)'# & ~NESTBugFound & ~NearAnodeBugFound' & (abs(energy-2614)<10)'
s_filter='''
standoff>20   &\
m_DNNvalue>0  &\
energy>0      &\
energy<3000   &\
passed_z_thresh &\
passed_xy_thresh &\
(n_x_ch_abovenoise>0) &\
(n_y_ch_abovenoise>0) &\
(m_nOPCal< (1.064*m_nQ+703))&\
(m_nOPCal> (0.644*m_nQ-2411))&\
~NESTBugFound &\
~NearAnodeBugFound       
         '''

In [ ]:
print("\ns_filter:\n",s_filter.replace('&','&\n'))

In [ ]:
print("\ndec102020_filter:\n",dec102020_filter.replace('&','&\n'))

In [ ]:
#isotopes order in the simulation
isotopes=['Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208']

In [ ]:
#read data file and return the related dataframe
def return_raw_df(file):
    '''
    returns the raw dataframe in the rootfile for the included tree
    '''
    f=uproot.open(file+':tree')
    df=f.arrays(f.keys(),library='pd')
    return df

#apply the given cut and get the cut dataframe
def return_cut_df(df,cut):
    '''
    Applies the `cut` with the `query` funciton in the provided `df` and returns a new dataframe
    '''
    return df.query(cut).reset_index(drop=True)


#counts the isotopes in the given dataframe and returns the respective number
def get_isotopes_counts(df,isotopes=isotopes):
    '''
    counts the number of respective `isotopes` in the given dataframe and returns a dataframe with `isotopes` and `counts'
    '''
    counts_df=pd.DataFrame(df.isotope.value_counts().reset_index().values,columns=['isonum','counts'])\
    .sort_values(by=['isonum'])\
    .reset_index(drop=True)

    counts_df["isotopes"]=counts_df.isonum.apply(lambda x:isotopes[x-1])
    counts_df=counts_df[['isonum','isotopes','counts']]
    #raw_counts.index=raw_counts.isotopes
    #print(raw_counts.to_string())
    return counts_df

#return a single dataframe with the g4tree_counts and cut counts

def get_both_counts(df_g4tree,df_cut):
    '''
    returns the single dataframe including the number of counts for `g4tree` as well as `cut` for provided `df_g4tree` and `df_cut' dataframes
    '''
    #pass
    df_g4tree.rename(columns={"counts":"g4tree_counts"},inplace=True)
    df_cut.rename(columns={"counts":"cut_counts"},inplace=True)
    merged=df_g4tree.merge(df_cut,how='left')
    merged=merged[['isotopes','g4tree_counts','cut_counts']]
    merged=merged.set_index('isotopes')
    merged.fillna(0,inplace=True)
    return merged

#returns the colum names in the given dataframe
def get_column_names(df):
    '''
    returns the column names in the `df` dataframe
    '''
    return df.columns

In [ ]:
#produces the energy spectrum
def get_energy_spectrum(df,loc,bins=1000,weights="weights"):
    '''returns the energy spectrum including the weights of isotopes'''
    
    sns.set(rc={'figure.figsize':(20,10)})
    
    #adding the isotop name column in the df dataframe
    df['iso_name']=df['isotope'].map({1:'Th228',2:'Ra224',3:'Rn220',4:'Po216',5:'Pb212',6:'Bi212',7:'Tl208'})
    
    
    #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
    g=sns.histplot(data=df, x="energy",hue="iso_name",weights="weight",bins=bins,element="step",fill=False)#,palette=['r','b','g','y','k'])
    #plt.legend(labels=iso_names)
    g.set_xlabel('Energy [keV]')
    g.set_ylabel('Counts')
    plt.yscale("log")
    plt.title(f'Energy specturm pdf for Th228 at {loc}')
    plt.show()


In [ ]:
#function to get the combined plot

def get_combined_count_plot(df,loc,save_fig=False, skipThreshold=0.1):
    '''
    return the plot of counts vs isotope numbers for the given dataframe
    '''
    plt.rcParams["figure.figsize"] = (16,12)
    ax=df.plot.bar()
    for c in ax.containers:
        ax.bar_label(c, fmt='%.0f', label_type='edge',rotation=90,color='k',fontweight='bold',padding=1.)
    plt.xlabel('ISOTOPES')
    plt.title(f'Isotope counts for 10M Th228 at {loc} for skipEThreshold: {skipEThreshold} keV \n')
    plt.ylabel('COUNTS')
    plt.yscale('log')
    if save_fig:
        fig_name=f'isotope_counts_{loc}.pdf'
        print(f"saving plot as: {fig_name}")
        plt.savefig(fig_name)
    plt.show()

In [ ]:
#function to return an energy spectrum and bar graph

def get_spectrum_and_bar(df,merged,loc,bins=1000,save_fig=False):
    '''
    returns an energy pdf and bar graph for given dataframe `df` and combined count dataframe `merged`
    '''
    fig, axes = plt.subplots(1, 2,gridspec_kw={'width_ratios':[3,1]})
    sns.set(rc={'figure.figsize':(20,10)})
    
    #mapping the isotope number to respective name
    df['iso_name']=df['isotope'].map({1:'Th228',2:'Ra224',3:'Rn220',4:'Po216',5:'Pb212',6:'Bi212',7:'Tl208'})
    #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
    g=sns.histplot(data=df, x="energy",hue="iso_name",weights="weight",bins=1000,element="step",fill=False,ax=axes[0])#,palette=['r','b','g','y','k'])
    #plt.legend(labels=iso_names)
    g.set_xlabel('Energy [keV]')
    g.set_ylabel('Counts')
    g.set_yscale("log")

    #g1=sns.barplot(data=merged,x='
    ax=merged[["g4tree_counts","cut_counts"]].plot.bar(ax=axes[1])
    for c in ax.containers:
        # set the bar label
        ax.bar_label(c, fmt='%.0f', label_type='edge',fontweight='bold')
    ax.set_xlabel('Isotopes')

    ax.set_ylabel('Counts')
    ax.set_yscale('log')

    plt.suptitle(f'Energy spectrum and event counts for 10M Th228 at {loc} for skipEThreshold: {skipEThreshold} keV \n')
    if save_fig:
        file_name='energyandisotopes.pdf'
        print(f"saving the graph to the file {file_name}\n")
        plt.savefig(file_name)

    plt.show()

In [ ]:
#g4tree dataframe
df_g4tree=return_raw_df(file_nx)
df_g4tree

In [ ]:
#column names
get_column_names(df_g4tree)

In [ ]:
#cut dataframe
cut=s_filter

print("\ncut-information:\n",cut.replace("&","&\n"))
df_cut=return_cut_df(df_g4tree,cut)
df_cut

In [ ]:
#isotopes counts
g4tree_counts=get_isotopes_counts(df_g4tree)  #g4tree counts

cut_counts=get_isotopes_counts(df_cut)        #cut counts



In [ ]:
g4tree_counts

In [ ]:
cut_counts


In [ ]:
#both counts df
combined_counts=get_both_counts(g4tree_counts,cut_counts)
combined_counts

In [ ]:
pr_line='\n'+30*'-'+'\n'
print(pr_line)
br_list=[1,1,1,1,1,1,0.359] #branching ratio of the isotopes
print(f"\nIsotopes and Branching order\n")
for i,j in zip(isotopes,br_list):
    print(f"{i}\t{j}")
print(pr_line)

In [ ]:
#resulting dataframe after applicaiton of br 
combined_counts['cut_counts']=combined_counts['cut_counts']*br_list
combined_counts['cut_counts']=combined_counts['cut_counts'].astype(int)

In [ ]:
#print(merged[['cut_counts','s_counts']].to_string())
combined_counts

In [ ]:
s_counts=[0,0,6,12,0,13570,234244]  #count based on scott's count for px position

In [ ]:
combined_counts["s_counts"]=s_counts

In [ ]:
combined_counts

# ISOTOPES COUNTS BAR DIAGRAM

In [ ]:
#give the name of the combined dataframe name and location
loc='NX'
save_fig=False
df=combined_counts

get_combined_count_plot(df,loc=loc,save_fig=save_fig)

# ENERGY PDF AND BAR GRAPHS

In [ ]:
loc='NX'
save_fig=True
get_spectrum_and_bar(df_cut,combined_counts,loc,save_fig=save_fig)

In [ ]:
iso=['Th228','Ra224','Rn220','Po216','pb212','bi212','tl208']
col=['b','y','g','k','c','m','r']
filtered_th228=df_cut.copy()
print(f"Total rows: {filtered_th228.shape[0]}")
for i in [1,2,3,4,5,6,7]:
    #print(non_filtered_th228_px.head())
    df_new=filtered_th228[filtered_th228.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;weight=df_new.weight
    plt.hist(px_energy,2000,density=False,weights=weight,histtype='step',alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Raw count ')
    plt.legend()
plt.title("SS Energy for Th228 at all positions")
plt.show()

In [ ]:
# more bins testing
# for i in [500,1000,1500,2000]:
#     get_energy_spectrum(df_cut,loc='NX',bins=i)

In [ ]:
# #plot for combined
# %matplotlib inline
# skipEThreshold=0.1
# # merged=merged[['isotopes','raw_counts','cut_counts']]
# # merged=merged.set_index('isotopes')
# ax=combined_counts.plot.bar()
# for c in ax.containers:
#     # set the bar label
#     ax.bar_label(c, fmt='%.0f', label_type='edge')
# plt.xlabel('''ISOTOPES''')
# plt.title(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV \n')
# plt.ylabel('COUNTS')
# plt.yscale('log')
# txt='test'
# #plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
# #plt.text(2.5, 10e4, 'cut: '+s_filter.strip().replace(" ",""), ha='center',fontsize='xx-small',color='r',fontweight='bold')

# plt.show()

In [ ]:
# ax=merged.plot.bar(y='raw_counts')
# plt.yscale('log')

In [ ]:
# #plot for raw data
# skipEThreshold=0.1
# plt.figure(figsize=(16,10))
# raw_counts=raw_counts.set_index('isotopes')
# ax=raw_counts.plot.bar()
# for c in ax.containers:
#     # set the bar label
#     ax.bar_label(c, fmt='%.0f', label_type='edge')
# plt.xlabel('ISOTOPES')
# plt.title(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV')
# plt.ylabel('COUNTS')
# plt.yscale('log')
# plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
# plt.show()

In [ ]:
# #plot for cut data
# skipEThreshold=0.1
# plt.figure(figsize=(16,10))
# cut_counts=cut_counts.set_index('isotopes')
# ax=cut_counts.plot.bar()
# for c in ax.containers:
#     # set the bar label
#     ax.bar_label(c, fmt='%.0f', label_type='edge')
# plt.xlabel('ISOTOPES')
# plt.title(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV')
# plt.ylabel('COUNTS')
# plt.yscale('log')
# #plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
# plt.show()

In [ ]:
#energy graph
# df_iso=df_s.query("isotope==7")`
# df_iso.energy.plot()
#df_iso.plot('energy','weight')
# ene=df_iso.energy
# weight=df_iso.weight
# label_value="test"
#out=ax.hist(df_iso.energy, bins=200, histtype=u'step', weights=weight, density=True, label=label_value)
# plt.show()
#df_s.columns

In [ ]:
#sns.relplot(data=df_s,x='energy',y='weight',hue='isotope')
# px_energy=df_cut.energy
# plt.hist(px_energy,200,density=False,histtype='step',facecolor='g',alpha=0.75)
# plt.yscale('log')
# plt.xlabel('Energy [keV]')
# plt.ylabel('Filtered count ')
# plt.show()

In [ ]:
# iso_names=cut_counts.isonum.apply(lambda x:isotopes[x-1])
# iso_names

In [ ]:
# bins=int(merged.cut_counts.sum())
# bins

In [ ]:
#iso_name=['Th228','Ra224','Rn220','Po216','pb212','bi212','tl208']

In [ ]:
#df_cut['iso_name']=df_cut['isotope'].map({1:'Th228',2:'Ra224',3:'Rn220',4:'Po216',5:'Pb212',6:'Bi212',7:'Tl208'})

In [ ]:
#df_cut.head()

In [ ]:
# #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step")
# #bins=merged.cut_counts.sum().values
# sns.set(rc={'figure.figsize':(20,10)})
# #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
# g=sns.histplot(data=df_cut, x="energy",hue="iso_name",weights="weight",bins=1000,element="step",fill=False)#,palette=['r','b','g','y','k'])
# #plt.legend(labels=iso_names)
# g.set_xlabel('Energy [keV]')
# g.set_ylabel('Count')
# plt.yscale("log")


In [ ]:
# df_s=df_cut.copy()
# merged=combined_counts
# loc='NX'
# fig, axes = plt.subplots(1, 2,gridspec_kw={'width_ratios':[3,1]})
# sns.set(rc={'figure.figsize':(20,10)})
# #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
# g=sns.histplot(data=df_s, x="energy",hue="iso_name",weights="weight",bins=1000,element="step",fill=False,ax=axes[0])#,palette=['r','b','g','y','k'])
# #plt.legend(labels=iso_names)
# g.set_xlabel('Energy [keV]')
# g.set_ylabel('Counts')
# g.set_yscale("log")

# #g1=sns.barplot(data=merged,x='
# ax=merged[["g4tree_counts","cut_counts"]].plot.bar(ax=axes[1])
# for c in ax.containers:
#     # set the bar label
#     ax.bar_label(c, fmt='%.0f', label_type='edge')
# ax.set_xlabel('Isotopes')

# ax.set_ylabel('Counts')
# ax.set_yscale('log')

# plt.suptitle(f'Energy spectrum and event counts for 10M Th228 at {loc} for skipEThreshold: {skipEThreshold} keV \n')
# plt.savefig('energyandisotopes.pdf')

# plt.show()